In [1]:
#import packages
import pandas as pd
import os

In [2]:
#import csvs
file_name = os.path.join("Resources", "students_complete.csv")
student_data = pd.read_csv(file_name)

file_name_2 = os.path.join("Resources", "schools_complete.csv")
school_data = pd.read_csv(file_name_2)

# District Summary

In [45]:
#Count Schools
total_schools = school_data['name'].count()

#Count Students
total_students = student_data['name'].count()

#Sum School Budgets
total_budget = school_data['budget'].sum()

#Find Average Reading Score
avg_reading_score = student_data['reading_score'].mean()

#Find Average Math Score
avg_math_score = student_data['math_score'].mean()

#Find Percentage of Passing Reading Scores (Passing is grade > 70%)
pct_pass_reading = (student_data['name'][(student_data["reading_score"] >= 70)].count()/total_students)*100

#Find Percentage of Passing Math Scores (Passing is grade > 70%)
pct_pass_math = (student_data['name'][(student_data["math_score"] >= 70)].count()/total_students)*100

#Calculate Overall Pass Rate (Averaging Reading and Math Percentages)
overall_pass_rate = (pct_pass_reading + pct_pass_math)/2

#Put Statistics into dataframe
district_summary = pd.DataFrame({"Total Schools": [total_schools],
                                  "Total Students": [total_students],
                                  "Total Budget": [total_budget],
                                  "Avg Math Score": [avg_math_score],
                                  "Avg Reading Score": [avg_reading_score],
                                  "% Passing Math": [pct_pass_math],
                                  "% Passing Reading": [pct_pass_reading],
                                  "% Overall Passing Rate": [overall_pass_rate]
                                  })

#Format Total budget with $ and commas
district_summary['Total Budget'] = district_summary['Total Budget'].map("${:,.0f}".format)

#Reorder Columns
district_summary = district_summary[["Total Schools", "Total Students", "Total Budget", "Avg Math Score", "Avg Reading Score","% Passing Math", "% Passing Reading","% Overall Passing Rate"  ]]

#Show Results
district_summary

,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428",78.985371,81.87784,74.980853,85.805463,80.393158


# School Summary

In [47]:
#Create dataframe with Math Pass % from Student Data and rename column
schoolpct_pass_math = pd.DataFrame(student_data['math_score'][(student_data["math_score"] >= 70)].groupby(student_data['school']).count()/student_data['math_score'].groupby(student_data['school']).count()*100)
schoolpct_pass_math = schoolpct_pass_math.rename(columns={'math_score':'% Passing Math'})

#Create dataframe with Reading Pass % from Student Data and rename column
schoolpct_pass_reading = pd.DataFrame(student_data['reading_score'][(student_data["reading_score"] >= 70)].groupby(student_data['school']).count()/student_data['reading_score'].groupby(student_data['school']).count()*100)
schoolpct_pass_reading = schoolpct_pass_reading.rename(columns={'reading_score':'% Passing Reading'})

#Merge above dataframes and calculate overall pass rate
school_pass_pct = pd.merge(schoolpct_pass_math, schoolpct_pass_reading, how='outer', left_index=True, right_index=True)
school_pass_pct['Overall Pass Rate']=(school_pass_pct['% Passing Reading']+school_pass_pct['% Passing Math'])/2

#Group data by school and delete unnecessary column
student_school_data = student_data.groupby(['school'])
school_avgs = student_school_data.mean()
del school_avgs['Student ID']

#Merge overall pass rate with pass %'s
combined_school_data = pd.merge(school_pass_pct, school_avgs, how='outer', left_index=True, right_index=True)
combined_school_data = combined_school_data.rename(columns={'reading_score':'Avg Reading Score','math_score':'Avg Math Score'})

#Create dataframe with above data
school_summary = pd.DataFrame({"School Name": school_data['name'],
                                  "School Type": school_data['type'],
                                   "Total Students": school_data['size'],
                                  "Total School Budget": school_data['budget'],
                                  "Per Student Budget": school_data['budget']/school_data['size'],
                                  })
#Set school name as index
school_summary = school_summary.set_index("School Name")
school_summary = pd.merge(school_summary, combined_school_data, how='outer', left_index=True, right_index=True)

#Format columns with dollar values with $ and commas
school_summary['Total School Budget'] = school_summary['Total School Budget'].map("${:,.0f}".format)
school_summary['Per Student Budget'] = school_summary['Per Student Budget'].map("${:,.0f}".format)

#Reorder columns
school_summary = school_summary[["School Type","Total Students","Total School Budget","Per Student Budget", "Avg Math Score", "Avg Reading Score", "% Passing Math", "% Passing Reading", "Overall Pass Rate"  ]] 

#Display data
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
Bailey High School,District,4976,"$3,124,928",$628,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,"$1,081,356",$582,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,"$1,884,411",$639,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,"$1,763,916",$644,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,"$917,500",$625,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,4635,"$3,022,020",$652,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087",$581,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,2917,"$1,910,635",$655,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650",$650,77.072464,80.966394,66.057551,81.222432,73.639992
Pena High School,Charter,962,"$585,858",$609,83.839917,84.044699,94.594595,95.945946,95.270270


# Top Performing Schools (By Passing Rate)

In [48]:
#Select five highest overall pass rates
top_5 = school_summary.nlargest(5, 'Overall Pass Rate')

#Display data
top_5 

,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
Cabrera High School,Charter,1858,"$1,081,356",$582,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130",$638,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858",$609,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500",$625,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574",$578,83.274201,83.989488,93.867718,96.539641,95.203679


# Bottom Performing Schools (By Passing Rate)

In [49]:
#Select five lowest overall pass rates
bottom_5 = school_summary.nsmallest(5, 'Overall Pass Rate')

#Display data
bottom_5

,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
Rodriguez High School,District,3999,"$2,547,363",$637,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411",$639,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635",$655,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650",$650,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916",$644,77.102592,80.746258,68.309602,79.299014,73.804308


# Math Scores by Grade

In [56]:
#Group data by grades
data_by_grade = student_data.groupby(['school','grade']).mean()

#Rename columns
math_by_grade = pd.DataFrame(data_by_grade['math_score']).rename(columns={'math_score':'Avg Math Score'})

#Move grades to columns
math_by_grade.unstack()

Avg Math Score                                 
grade                           10th       11th       12th        9th
school                                                               
Bailey High School         76.996772  77.515588  76.492218  77.083676
Cabrera High School        83.154506  82.765560  83.277487  83.094697
Figueroa High School       76.539974  76.884344  77.151369  76.403037
Ford High School           77.672316  76.918058  76.179963  77.361345
Griffin High School        84.229064  83.842105  83.356164  82.044010
Hernandez High School      77.337408  77.136029  77.186567  77.438495
Holden High School         83.429825  85.000000  82.855422  83.787402
Huang High School          75.908735  76.446602  77.225641  77.027251
Johnson High School        76.691117  77.491653  76.863248  77.187857
Pena High School           83.372000  84.328125  84.121547  83.625455
Rodriguez High School      76.612500  76.395626  77.690748  76.859966
Shelton High School        82.917411  83.383495  83.778976  83.420755
Thomas High School         83.087886  83.498795  83.497041  83.590022
Wilson High School         83.724422  83.195326  83.035794  83.085578
Wright High School         84.010288  83.836782  83.644986  83.264706

# Reading Scores by Grade

In [44]:
#Rename columns
reading_by_grade = pd.DataFrame(data_by_grade['reading_score']).rename(columns={'reading_score':'Avg Reading Score'})

#Move grades to columns
reading_by_grade.unstack()

Avg Reading Score                                 
grade                              10th       11th       12th        9th
school                                                                  
Bailey High School            80.907183  80.945643  80.912451  81.303155
Cabrera High School           84.253219  83.788382  84.287958  83.676136
Figueroa High School          81.408912  80.640339  81.384863  81.198598
Ford High School              81.262712  80.403642  80.662338  80.632653
Griffin High School           83.706897  84.288089  84.013699  83.369193
Hernandez High School         80.660147  81.396140  80.857143  80.866860
Holden High School            83.324561  83.815534  84.698795  83.677165
Huang High School             81.512386  81.417476  80.305983  81.290284
Johnson High School           80.773431  80.616027  81.227564  81.260714
Pena High School              83.612000  84.335938  84.591160  83.807273
Rodriguez High School         80.629808  80.864811  80.376426  80.993127
Shelton High School           83.441964  84.373786  82.781671  84.122642
Thomas High School            84.254157  83.585542  83.831361  83.728850
Wilson High School            84.021452  83.764608  84.317673  83.939778
Wright High School            83.812757  84.156322  84.073171  83.833333

# Score by School Spending

In [57]:
#Create bins and bin names
budget_bins = [0, 585, 610, 645, 656]
budget_bin_names = ['Low','Low-Medium', 'High-Medium', 'High']

#Recalculate Per student budget to reformat column
df = school_data['budget']/school_data['size']
school_summary['Per Student Budget']=df.values

#Add bin categories to dataframe and groupby them
school_summary['Budget Size'] = pd.cut(school_summary['Per Student Budget'], budget_bins, labels=budget_bin_names)
school_summary_budgetgroups = school_summary.groupby('Budget Size')

#Select Columns
school_summary_budgetgroups = school_summary_budgetgroups['Avg Math Score','Avg Reading Score','% Passing Math', '% Passing Reading','Overall Pass Rate']

#Display data
school_summary_budgetgroups.max()

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
Budget Size,,,,,
Low,83.803279,83.814988,92.505855,96.252927,94.379391
Low-Medium,83.839917,84.044699,94.594595,95.945946,95.270270
High-Medium,83.682222,83.989488,94.133477,97.138965,95.586652
High,83.418349,83.848930,93.272171,97.308869,95.290520


# Score by School Size

In [11]:
#Create bins and bin names
size_bins = [0, 2000, 3000, 5000]
size_bin_names = ['Small','Medium', 'Large' ]

#Add bin categories to dataframe and groupby them
school_summary['School Size'] = pd.cut(school_summary['Total Students'], size_bins, labels=size_bin_names)
school_summary_sizegroups = school_summary.groupby('School Size')

#Select Columns
school_summary_sizegroups = school_summary_sizegroups['Avg Math Score','Avg Reading Score','% Passing Math', '% Passing Reading','Overall Pass Rate']

#Display data
school_summary_sizegroups.max()

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
School Size,,,,,
Small,83.839917,84.044699,94.594595,97.308869,95.586652
Medium,83.274201,83.989488,93.867718,96.539641,95.203679
Large,77.289752,81.033963,66.752967,81.933280,74.306672


# Score by School Type

In [12]:
#Groupby School Type
school_summary_typegroups = school_summary.groupby('School Type')

#Select Columns
school_summary_typegroups = school_summary_typegroups['Avg Math Score','Avg Reading Score','% Passing Math', '% Passing Reading','Overall Pass Rate']

#Display Data
school_summary_typegroups.max()

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
School Type,,,,,
Charter,83.839917,84.044699,94.594595,97.308869,95.586652
District,77.289752,81.182722,68.309602,81.933280,74.306672
